### TRAINING MODELS

In [2]:
# IMPORT EVERYTHING
from Architectures.AE_ADJ_v2 import AE

from Data.load import load, load_middle_slices
from Visualizations.latent_space_projections import plot_multiple_datasets
from Visualizations.plots import plot_middle_slices_in_range, plot_models_training_time, compare_models_loss_history, plot_images, compare_models_reconstruction
from Metrics.metrics_tf import MSE_loss, NMSE_loss, NRMSE_loss, SSIM_loss
from Metrics.metrics import NMSE, SSIM, NRMSE, MSE

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap, TSNE
from sklearn.preprocessing import StandardScaler

from tensorflow.keras import layers, Model, activations, regularizers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from skimage.metrics import structural_similarity as ssim

import umap
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import pandas as pd
import seaborn as sns
import glob
import tensorflow as tf
import time
import math
import nibabel as nib
import json
from tqdm import tqdm

c:\Users\kiran\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
groups = {}
#groups3D = {}
for group in tqdm(["CN","MCI","AD"],"Opening faster"):
    groups[group]=np.load(open(f"Data/D2-{group}.npy","rb"))
    #groups3D[group]=np.load(open(f"Data/D3-{group}.npy","rb"))

Opening faster: 100%|██████████| 3/3 [00:00<00:00, 29.39it/s]


In [ ]:
"""
A2 = AE(name="A2", input_shape=groups["CN"].shape, latent_dim=16,
    encoder_layers=[
        (layers.Conv2D, 16, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
        (layers.Conv2D, 32, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
    ],
    decoder_layers=[
        (layers.Conv2D, 32, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 16, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
        (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
    ], VAE_model=False, optimizer="adam", loss_fn="mse").compile_model()
"""

"""
models_list = [
    AE(name="public_repo_2d_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (2, 2)}),
            (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 128, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization()
        ],
        decoder_layers=[
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 64, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same',}),
        ],
        VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),
    
    AE(name="public_unsupervised_anomaly_2d_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'strides': 2, 'padding': 'same'}), 
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2D, 16, (1, 1), {'activation': 'relu', 'strides': 1, 'padding': 'same'}), 
            (layers.Conv2D, 128, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 64, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 32, (5, 5), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 1, (1, 1), {'activation': 'sigmoid', 'padding': 'same'})
        ], VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),

    AE(name="public_frontiers_paper_2d_mse", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=MSE_loss).compile_model(),

    AE(name="public_alcoholism_paper_2D_ssim", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 40, (5, 5), {'activation': 'relu', 'strides': (3, 3), 'padding': 'valid'}), layers.BatchNormalization(),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'strides': (5, 5), 'padding': 'valid'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'strides': (1, 1), 'padding': 'same'}), layers.BatchNormalization(), (layers.MaxPooling2D, {'pool_size': (3, 3), 'strides': (1, 1), 'padding': 'same'}),
        ],
        decoder_layers=[
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 120, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 80, (3, 3), {'activation': 'relu', 'padding': 'same'}), layers.BatchNormalization(), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2D, 40, (5, 5), {'padding': 'same'}), 
            (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
        ], VAE_model=False, optimizer="adam", loss_fn=MSE_loss).compile_model(),
]
"""
#  layers.ReLU(), layers.LeakyReLU(alpha=0.2),

models_list = [
    AE(name="public_frontiers_paper_2d_mse_lrED", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'leaky_relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'leaky_relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'leaky_relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'leaky_relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=MSE_loss).compile_model(),
    
    AE(name="public_frontiers_paper_2d_mse_lrE", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), (layers.MaxPooling2D, {'pool_size': (2, 2), 'strides': (2, 2), 'padding': 'same'}),
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (2, 2), 'padding': 'same'}), 
            (layers.Conv2D, 32, (3, 3), {'activation': 'leaky_relu', 'strides': (1, 1), 'padding': 'same'}), 
        ],
        decoder_layers=[
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same', 'strides': 2}),
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), 
            (layers.Conv2DTranspose, 32, (3, 3), {'activation': 'relu', 'padding': 'same'}), (layers.UpSampling2D, {'size': (2, 2)}),
            (layers.Conv2DTranspose, 1,  (3, 3), {'activation': 'sigmoid','padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=MSE_loss).compile_model(),
    
    AE(name="custom_simplest_model", input_shape=groups["CN"].shape, latent_dim=128,
        encoder_layers=[
            (layers.Conv2D, 4, (3,3), {'strides': 2, 'padding': 'same', 'activation': 'leaky_relu'}),
        ],
        decoder_layers=[
            (layers.Conv2D, 4, (3,3), {'padding': 'same'}), (layers.UpSampling2D, (2,2)),
            (layers.Conv2D, 1, (3,3), {'padding': 'same'}),
        ], VAE_model=False,  optimizer="adam", loss_fn=MSE_loss).compile_model(),
]

Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 20, 24, 32)
Interpreted image shape: (80, 96, 1) Pre-flattened latent shape: (None, 40, 48, 4)


In [7]:
for m in models_list: #[A2,A4,A5]
    m.train(groups["CN"], epochs=1000, batch_size=128, verbose=True, 
    save_path=r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images", 
    save_interval=100, patience=100)

Training public_frontiers_paper_2d_mse_lrED on CPU
Input shape: (988, 80, 96)
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 713ms/step - loss: 0.1093 - val_loss: 0.0606
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 669ms/step - loss: 0.0479 - val_loss: 0.0207
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 641ms/step - loss: 0.0209 - val_loss: 0.0152
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 701ms/step - loss: 0.0162 - val_loss: 0.0139
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 684ms/step - loss: 0.0153 - val_loss: 0.0131
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 691ms/step - loss: 0.0146 - val_loss: 0.0131
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 690ms/step - loss: 0.0143 - val_loss: 0.0127
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 671ms/step - loss: 0.0140 - val_loss: 0.0127
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 680ms/step - loss: 0.0139 - val_loss: 0.0126
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 666ms/step - loss: 0.0139 - val_loss: 0.0124
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 688ms/step - loss: 0.0138 - val_loss: 0.0125
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 662ms/step - loss: 0.0136 - val_loss: 0.0125
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 665ms/step - loss:

In [ ]:
# Test to see if loading works
# models_list = [AE.open(r"C:\Users\kiran\Documents\_UIS\sem6\BACH\DementiaMRI\AE_ADJ_models\newer_images\public_repo\epoch_9")]

[<AE name=public_frontiers_paper_2d_mse_lrED, built=True>,
 <AE name=public_frontiers_paper_2d_mse_lrE, built=True>,
 <AE name=custom_simplest_model, built=True>]